# Description  

Every day, *ABC* company's risk division assigns a fraud risk score to all of our payments customers. Our customers are all merchants who accept credit card payments for their business. Some of the applications we receive to process payments are from people trying to commit fraud by pretending to be legitimate merchants. Typically, these fraudulent merchants will use stolen credit cards to funnel money into their account.  

The zipped sample data contains a few of the types of data we use in our assessment for fraud.  All of the merchants in the attached files have made mobile credit card payment transactions at least once.  Some of these merchants turned out later to be fraudulent. 

The zipped file includes three tables from merchants who opened an account between 2013 and July 2015.  For the last 3 weeks of data (July 10, 2015 to July 31, 2015) there is no information regarding whether the merchant was closed for fraud.  For all accounts opened before July 10, information on when the account was closed for fraud is provided.  


In [1]:
import pandas as pd
import numpy as np

In [4]:
app_dat = pd.read_csv('./../data/training exercise data - application.csv')
first_txns = pd.read_csv('./../data/training exercise data - first transaction.csv')
trgt_dat = pd.read_csv('./../data/training exercise data - fraud target.csv')

In [17]:
app_dat.sample(3)

,merchantidentifier,IP_Address_organization,IP_Address_Location,Pierced_IP_address_organization,Pierced_IP_address_location,timezone_differencefromGMT_using_application_device,date_account_opened,state_from_application_address,AreaCodeState
1584,33243117,ROAD RUNNER,CALIFORNIA,ROAD RUNNER,CALIFORNIA,480.0,2013-03-13,CA,CA
87960,44866773,SUDDENLINK COMMUNICATIONS,LOUISIANA,SUDDENLINK COMMUNICATIONS,LOUISIANA,360.0,2014-03-08,LA,LA
112885,19899755151,AT&T U-VERSE,CALIFORNIA,AT&T U-VERSE,CALIFORNIA,480.0,2015-04-28,CA,CA


In [12]:
first_txns.sample(3)

,merchantidentifier,first_transaction_time,first_transaction_amount
44273,36136195,2013-11-14 07:19:58,3.0
114649,19967450487,2014-09-06 21:34:21,1.0
118760,19953794909,2015-03-30 09:49:03,50.0


In [23]:
trgt_dat.sample(3)

,merchantidentifier,dateclosedforfraud,Flag_for_data_with_fraud_outcomes
113454,19914568767,NaN,1
21957,30223171,NaN,1
65957,19963069691,2015-04-02,1


In [20]:
(app_dat.shape, first_txns.shape, trgt_dat.shape)

((132552, 9), (132552, 3), (132552, 3))

# Data Dictionary 

**Application Data**  
* Merchant Identifier: A unique ID that allows you to link between tables.
* IP_Address__Organization:  The provider associated with the IP Address.
* IP_Address__State:  The state associated with the IP Address.
* Pierced_IP_Address__Organization:  Some real and fraudulent merchants have masked IP Address—the “pierced” IP Address Organization is the one associated with the underlying IP which may differ from the one above.   
* Pierced_IP_Address__State:  Some real and fraudulent merchants have masked IP Address—the “pierced” IP Address State is the location associated with the underlying IP which may differ from the one above.   
* Timezone_differencefromGMT_using_application_device:  This provides the timezone set in the device based on the difference between the device time and Greenwich Mean Time.
* Date_account_opened: The date when the account was opened with Intuit.
* State_from_application_address:	The state provided in the business address by the applicant.
* AreaCodeState:  The state associated with the area code provided in the business phone number at time of application.  

**First Transaction Data** 
This table contains data from the first mobile transaction of the merchant including the following:  

* Merchant Identifier: A unique ID that allows you to link between tables.
* First_Transaction_Time:  The date and time of the first transaction by the merchant (in local transaction time for the merchant’s location).
* First_Transaction_Amount:  The dollar amount of the first transactio

**Fraud Target Data Table:**  
This table contains data on the target you are trying to forecast:  

* Merchant Identifier: A unique ID that allows you to link between tables.
* Dateclosedforfraud:  This is the date when a merchant was closed for fraud.  If a merchant has a blank or null value here and the flag below has a value of 1, you can assume that the merchant was not closed for fraud.
* Flag_for_data_with_fraud_outcomes:  This flag separate merchants with outcomes data from those without outcomes data.  Merchants opened between July 10, 2015 and July 31, 2015 have that flag set to 0 and no fraud data available (remember, this is what you are trying to predict!)  For merchants opened before July 10, the flag is set to 1 and if the merchant was closed for fraud the date is shown.

# Objective  
Your goal is to assess the risk that an account will be closed for fraud within the first 60 days after the account is opened.  Specifically, we want you to identify the 100 merchants opened between July 10, 2015 and July 31, 2015 that you consider to be of greatest risk for being closed for fraud within 60 days of their open date.  

---

# Data Cleaning  

Let's first get all the data in one place. The tables are nicely linked by Merchant Id, and there seems to be a 1-1 correspondence between these, going by the similar number of records in each. Let's *left join* the first transaction data and the fraud target data to the applications table to get all the data in one place. The left join will help preserve all the rows in the applicationt able that might not match (although I suspect this isn't the case).

In [105]:
dat = app_dat.merge(first_txns, how='left', left_on='merchantidentifier', right_on='merchantidentifier').merge(trgt_dat, how='left', left_on='merchantidentifier', right_on='merchantidentifier')

In [106]:
dat.shape

(132552, 13)

Looks like a clean join. Let's be sure by counting the number of unique mechant identifiers before and after the joins. It looks like we're okay.

In [107]:
len(app_dat.merchantidentifier.unique()), len(dat.merchantidentifier.unique())

(132552, 132552)

In [111]:
dat.sample(5)

,merchantidentifier,IP_Address_organization,IP_Address_Location,Pierced_IP_address_organization,Pierced_IP_address_location,timezone_differencefromGMT_using_application_device,date_account_opened,state_from_application_address,AreaCodeState,first_transaction_time,first_transaction_amount,dateclosedforfraud,Flag_for_data_with_fraud_outcomes
71214,19956623321,COX COMMUNICATIONS,VIRGINIA,COX COMMUNICATIONS,VIRGINIA,300.0,2015-06-02,VA,VA,2015-07-10 06:39:21,2525.0,NaN,1
97977,19898491053,CHARTER COMMUNICATIONS,MICHIGAN,CHARTER COMMUNICATIONS,MICHIGAN,300.0,2014-06-06,MI,MI,2014-06-24 08:57:11,35.0,NaN,1
96598,19833253333,CHARTER COMMUNICATIONS,CALIFORNIA,CHARTER COMMUNICATIONS,CALIFORNIA,480.0,2014-06-28,CA,CA,2015-02-06 17:16:16,75.0,NaN,1
4039,27318791,VIASAT COMMUNICATIONS,NaN,VIASAT COMMUNICATIONS,NaN,360.0,2013-11-12,MS,MS,2013-11-16 07:23:48,10.0,NaN,1
97065,19863390831,COMCAST CABLE,COLORADO,COMCAST CABLE,COLORADO,420.0,2015-03-25,CO,NJ,2015-04-02 18:18:40,1.0,NaN,1


## Outcome Variable  

We'll create a new indicator specifically identifying an account as closed for fraud only if the following condition holds true: *Flag_for_data_with_fraud_outcomes* == 1 **and** *dateclosedforfraud* is not NULL

In [112]:
dat['is_fraud'] = np.where(((dat['Flag_for_data_with_fraud_outcomes']==1) & (dat['dateclosedforfraud'].notnull())), 1, 0)

For our missing value analysis we only need merchants with accounts opened before July 10, 2015. So let's subset our data to these accounts only. 

In [113]:
#making sure it's okay to subset using this flag
dat[dat['Flag_for_data_with_fraud_outcomes']==0]['first_transaction_time'].min()

'2015-07-10 00:27:38'

In [114]:
dat_train = dat[dat['Flag_for_data_with_fraud_outcomes']==1]

Let's define our *True Positives* as cases identified as fraudulent. What proportion of the data is this?

In [115]:
print '{:03.2f}% accounts from the entire dataset have been closed for fraudulent activity'.format((dat_train[dat_train['is_fraud']==1].shape[0]/float(dat_train.shape[0]))*100)

2.88% accounts from the entire dataset have been closed for fraudulent activity


As is expected in this domain, we have an **imbalanced dataset.** This entails looking into strategies to deal with imabalanced classes such as:  
* Synthetic Minority Oversampling  
* Cost based penalization  
* Weighted decision trees 

## Missing Values  
Before moving forward, let's assess what can be done about missing values in the data.

In [116]:
dat_train.isnull().sum()

merchantidentifier                                          0
IP_Address_organization                                     4
IP_Address_Location                                      5445
Pierced_IP_address_organization                             4
Pierced_IP_address_location                              5436
timezone_differencefromGMT_using_application_device      1369
date_account_opened                                         0
state_from_application_address                              0
AreaCodeState                                               0
first_transaction_time                                      0
first_transaction_amount                                    0
dateclosedforfraud                                     127117
Flag_for_data_with_fraud_outcomes                           0
is_fraud                                                    0
dtype: int64

In [117]:
#let's look at IP_Adress_organization
dat_train[dat_train.IP_Address_organization.isnull()] 
#NONE ARE MARKED FOR FRAUD!

,merchantidentifier,IP_Address_organization,IP_Address_Location,Pierced_IP_address_organization,Pierced_IP_address_location,timezone_differencefromGMT_using_application_device,date_account_opened,state_from_application_address,AreaCodeState,first_transaction_time,first_transaction_amount,dateclosedforfraud,Flag_for_data_with_fraud_outcomes,is_fraud
17046,27719017,NaN,NaN,NaN,NaN,360.0,2013-04-23,IL,IL,2013-04-30 20:03:11,135.69,NaN,1,0
19120,34677827,NaN,NaN,NaN,NaN,480.0,2013-04-24,WA,WA,2013-04-29 12:17:46,1.00,NaN,1,0
20992,27067827,NaN,NaN,NaN,NaN,300.0,2013-05-16,NY,NY,2013-06-10 13:45:16,1.00,NaN,1,0
71403,19985710875,NaN,NaN,NaN,NaN,360.0,2014-08-13,NY,NY,2015-05-22 18:22:12,475.00,NaN,1,0


In [118]:
#sample of IP_Address_Location being NULL
dat_train[dat_train.IP_Address_Location.isnull()].sample(5)

,merchantidentifier,IP_Address_organization,IP_Address_Location,Pierced_IP_address_organization,Pierced_IP_address_location,timezone_differencefromGMT_using_application_device,date_account_opened,state_from_application_address,AreaCodeState,first_transaction_time,first_transaction_amount,dateclosedforfraud,Flag_for_data_with_fraud_outcomes,is_fraud
81656,43706539,VERIZON WIRELESS,NaN,VERIZON WIRELESS,NaN,360.0,2013-10-25,TX,TX,2013-11-19 19:04:20,50.00,NaN,1,0
8671,29032329,T-MOBILE USA,NaN,T-MOBILE USA,NaN,360.0,2013-08-10,IN,IN,2013-08-10 13:07:06,300.00,2013-08-28,1,1
15829,36796015,HUGHES NETWORK SYSTEMS,NaN,HUGHES NETWORK SYSTEMS,NaN,360.0,2013-08-26,AL,AL,2013-09-07 12:05:54,40.00,NaN,1,0
55582,40949775,MARRIOTT - IADTC,NaN,MARRIOTT - IADTC,NaN,300.0,2013-07-25,VA,MD,2013-07-29 06:33:10,10.00,2013-07-30,1,1
16853,29442999,VERIZON WIRELESS,NaN,VERIZON WIRELESS,NaN,360.0,2013-03-14,OK,OK,2013-03-21 09:00:11,0.52,NaN,1,0


In [119]:
print '{:03.5f}% of unknown IP address locations were identified as fraudulent cases'.format(dat_train[((dat_train.IP_Address_Location.isnull())&(dat_train.is_fraud==1))].shape[0]/float(dat_train[dat_train.IP_Address_Location.isnull()].shape[0]))

0.04885% of unknown IP address locations were identified as fraudulent cases


In [120]:
print '{:03.5f}% of unknown Pierced IP address locations were identified as fraudulent cases'.format(dat_train[((dat_train.Pierced_IP_address_location.isnull())&(dat_train.is_fraud==1))].shape[0]/float(dat_train[dat_train.Pierced_IP_address_location.isnull()].shape[0]))

0.04875% of unknown Pierced IP address locations were identified as fraudulent cases


In [121]:
print '{:03.5f}% of unknown \'device timezone difference from GMT\' records were identified as fraudulent cases'.format(dat_train[((dat_train.timezone_differencefromGMT_using_application_device.isnull())&(dat_train.is_fraud==1))].shape[0]/float(dat_train[dat_train.timezone_differencefromGMT_using_application_device.isnull()].shape[0]))

0.01972% of unknown 'device timezone difference from GMT' records were identified as fraudulent cases


Missing IP Addresses, Pierced IPs, or corresponding IP address locations don't seem to be very good predictors of fraudulent cases. Nevertheless, instead of dropping them, it may be wiser to bucket all these NaNs as *Unknown* to be dummied in training later. Let's make these changes in the original dataframe with the non-outcomes data.

In [122]:
dat['IP_Address_organization']=np.where(dat['IP_Address_organization'].isnull(), 'Unknown',dat['IP_Address_organization'])
dat['Pierced_IP_address_organization']=np.where(dat['Pierced_IP_address_organization'].isnull(), 'Unknown',dat['Pierced_IP_address_organization'])
dat['IP_Address_Location']=np.where(dat['IP_Address_Location'].isnull(), 'Unknown',dat['IP_Address_Location'])
dat['Pierced_IP_address_location']=np.where(dat['Pierced_IP_address_location'].isnull(), 'Unknown',dat['Pierced_IP_address_location'])

Now, let's split out all the non-outcomes data into a separate test set and save it to a csv file. This is the set of records we want to make our predictions on. 

In [131]:
dat[dat['Flag_for_data_with_fraud_outcomes']==0].to_csv('./../data/test_submit.csv', index=False)

And save the remaining dataframe as the final cleaned dataset to be used for further exploration, and model building.

In [132]:
dat[dat['Flag_for_data_with_fraud_outcomes']==1].to_csv('./../data/train.csv', index=False)

Next, we move to exploring the data including:  
* Outlier analysis  
* Feature engineering  
* Correlation analysis  
* External data considerations  
See the Jupyter notebook [Data_Explore.ipynb](./Data_Explore.ipynb)  

---